In [1]:
!pip install 'h5py==2.10.0' --force-reinstall
!pip install tensorflow-gpu==1.15

!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -

!echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list

!sudo apt-get update

!sudo apt-get install edgetpu-compiler


     |████████████████████████████████| 2.9 MB 7.7 MB/s 
     |████████████████████████████████| 15.7 MB 75 kB/s 
  Attempting uninstall: six
    Found existing installation: six 1.15.0
    Uninstalling six-1.15.0:
      Successfully uninstalled six-1.15.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.5.0 requires h5py~=3.1.0, but you have h5py 2.10.0 which is incompatible.
tensorflow 2.5.0 requires numpy~=1.19.2, but you have numpy 1.21.1 which is incompatible.
tensorflow 2.5.0 requires six~=1.15.0, but you have six 1.16.0 which is incompatible.
google-col

     |████████████████████████████████| 411.5 MB 8.0 kB/s 
     |████████████████████████████████| 50 kB 7.4 MB/s 
     |████████████████████████████████| 3.8 MB 39.7 MB/s 
     |████████████████████████████████| 503 kB 43.9 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7553 sha256=2e35864943aac6c2d6ccbf8c1b5ab6d22319f11a60dc910f460fae7e0be00775
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.5.0
    Uninstalling tensorflow-estimator-2.5.0:
      Successfully uninstalled tensorflow-estimator-2.5.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.5.0
    Uninstalling tensorboard-2.5.0:
      Successfully uninstalled tensorboard-2.5.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Succe

In [15]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import DenseNet121, DenseNet169, DenseNet201, \
  InceptionResNetV2, InceptionV3, MobileNet, MobileNetV2, NASNetLarge, NASNetMobile,\
  ResNet101, ResNet101V2, ResNet152, ResNet152V2, ResNet50, ResNet50V2, \
  VGG16, VGG19, Xception 
import os
from functools import partial

In [14]:
def representative_data_gen(input_shape):
  for i in range(100):
    yield [np.random.random((1,)+input_shape).astype(np.float32)]

def testEdgeTPUCompilation(modelGenerationFunction, fn="test"):
  # Load model and save as h5 file
  m = modelGenerationFunction()

  if fn is None:
    fn = m.name

  print(f"Model name: {fn}")

  input_shape = m.layers[0].input_shape[0][1:]
  print(f"Input shape: ", input_shape)
  m.save(fn+".h5")

  rdg = partial(representative_data_gen, input_shape=input_shape)

  # Compile to TFLITE
  converter = tf.lite.TFLiteConverter.from_keras_model_file(fn+".h5")
  converter.optimizations = [tf.lite.Optimize.DEFAULT]
  converter.representative_dataset = rdg
  converter.inference_input_type = tf.uint8
  converter.inference_output_type = tf.uint8
  tflite_model = converter.convert()

  # Save the model.
  with open(fn+".tflite", 'wb') as f:
    f.write(tflite_model)

  # Compile to Edge TPU
  edge_log = os.popen(f"edgetpu_compiler {fn}.tflite").read()
  
  with open(f"{fn}_edgetpu_compilation.log","w+") as fp:
    fp.write(edge_log)
    print(edge_log)

  # Load log
  with open(f"{fn}_edgetpu.log") as fp:
    log = fp.read()
    print(log)

%time testEdgeTPUCompilation(MobileNetV2, None)

Model name: mobilenetv2_1.00_224
Input shape
INFO:tensorflow:Froze 262 variables.
INFO:tensorflow:Converted 262 variables to const ops.
Edge TPU Compiler version 16.0.384591198
Started a compilation timeout timer of 180 seconds.

Model compiled successfully in 1297 ms.

Input model: mobilenetv2_1.00_224.tflite
Input size: 3.80MiB
Output model: mobilenetv2_1.00_224_edgetpu.tflite
Output size: 4.06MiB
On-chip memory used for caching model parameters: 3.88MiB
On-chip memory remaining for caching model parameters: 3.81MiB
Off-chip memory used for streaming uncached model parameters: 0.00B
Number of Edge TPU subgraphs: 1
Total number of operations: 72
Operation log: mobilenetv2_1.00_224_edgetpu.log
See the operation log file for individual operation details.
Compilation child process completed within timeout period.
Compilation succeeded! 

Edge TPU Compiler version 16.0.384591198
Input: mobilenetv2_1.00_224.tflite
Output: mobilenetv2_1.00_224_edgetpu.tflite

Operator                       

In [16]:
%time testEdgeTPUCompilation(MobileNet, None)

17227776/17225924 [==============================] - 0s 0us/step
Model name: mobilenet_1.00_224
Input shape
INFO:tensorflow:Froze 137 variables.
INFO:tensorflow:Converted 137 variables to const ops.
Edge TPU Compiler version 16.0.384591198
Started a compilation timeout timer of 180 seconds.

Model compiled successfully in 1170 ms.

Input model: mobilenet_1.00_224.tflite
Input size: 4.35MiB
Output model: mobilenet_1.00_224_edgetpu.tflite
Output size: 4.46MiB
On-chip memory used for caching model parameters: 4.30MiB
On-chip memory remaining for caching model parameters: 3.40MiB
Off-chip memory used for streaming uncached model parameters: 0.00B
Number of Edge TPU subgraphs: 1
Total number of operations: 39
Operation log: mobilenet_1.00_224_edgetpu.log
See the operation log file for individual operation details.
Compilation child process completed within timeout period.
Compilation succeeded! 

Edge TPU Compiler version 16.0.384591198
Input: mobilenet_1.00_224.tflite
Output: mobilenet_1.0

In [17]:
%time testEdgeTPUCompilation(DenseNet121, None)

Model name: densenet121
Input shape
INFO:tensorflow:Froze 606 variables.
INFO:tensorflow:Converted 606 variables to const ops.
Edge TPU Compiler version 16.0.384591198
Started a compilation timeout timer of 180 seconds.

Model compiled successfully in 4392 ms.

Input model: densenet121.tflite
Input size: 7.95MiB
Output model: densenet121_edgetpu.tflite
Output size: 9.34MiB
On-chip memory used for caching model parameters: 7.04MiB
On-chip memory remaining for caching model parameters: 0.00B
Off-chip memory used for streaming uncached model parameters: 2.98MiB
Number of Edge TPU subgraphs: 1
Total number of operations: 384
Operation log: densenet121_edgetpu.log
See the operation log file for individual operation details.
Compilation child process completed within timeout period.
Compilation succeeded! 

Edge TPU Compiler version 16.0.384591198
Input: densenet121.tflite
Output: densenet121_edgetpu.tflite

Operator                       Count      Status

MAX_POOL_2D                    1  

In [18]:
%time testEdgeTPUCompilation(DenseNet169, None)

58548224/58541896 [==============================] - 1s 0us/step
Model name: densenet169
Input shape
INFO:tensorflow:Froze 846 variables.
INFO:tensorflow:Converted 846 variables to const ops.
Edge TPU Compiler version 16.0.384591198
Started a compilation timeout timer of 180 seconds.

Model compiled successfully in 7514 ms.

Input model: densenet169.tflite
Input size: 13.97MiB
Output model: densenet169_edgetpu.tflite
Output size: 16.51MiB
On-chip memory used for caching model parameters: 7.04MiB
On-chip memory remaining for caching model parameters: 1.50KiB
Off-chip memory used for streaming uncached model parameters: 8.59MiB
Number of Edge TPU subgraphs: 1
Total number of operations: 533
Operation log: densenet169_edgetpu.log
See the operation log file for individual operation details.
Compilation child process completed within timeout period.
Compilation succeeded! 

Edge TPU Compiler version 16.0.384591198
Input: densenet169.tflite
Output: densenet169_edgetpu.tflite

Operator       

In [19]:
%time testEdgeTPUCompilation(DenseNet201, None)

82526208/82524592 [==============================] - 1s 0us/step
Model name: densenet201
Input shape
INFO:tensorflow:Froze 1006 variables.
INFO:tensorflow:Converted 1006 variables to const ops.
Edge TPU Compiler version 16.0.384591198
Started a compilation timeout timer of 180 seconds.

Model compiled successfully in 9801 ms.

Input model: densenet201.tflite
Input size: 19.65MiB
Output model: densenet201_edgetpu.tflite
Output size: 23.27MiB
On-chip memory used for caching model parameters: 7.04MiB
On-chip memory remaining for caching model parameters: 2.00KiB
Off-chip memory used for streaming uncached model parameters: 15.17MiB
Number of Edge TPU subgraphs: 1
Total number of operations: 622
Operation log: densenet201_edgetpu.log
See the operation log file for individual operation details.
Compilation child process completed within timeout period.
Compilation succeeded! 

Edge TPU Compiler version 16.0.384591198
Input: densenet201.tflite
Output: densenet201_edgetpu.tflite

Operator    

In [20]:
%time testEdgeTPUCompilation(InceptionResNetV2, None)

225214464/225209952 [==============================] - 6s 0us/step
Model name: inception_resnet_v2
Input shape
INFO:tensorflow:Froze 898 variables.
INFO:tensorflow:Converted 898 variables to const ops.
Edge TPU Compiler version 16.0.384591198
Started a compilation timeout timer of 180 seconds.

Model compiled successfully in 12451 ms.

Input model: inception_resnet_v2.tflite
Input size: 55.31MiB
Output model: inception_resnet_v2_edgetpu.tflite
Output size: 55.86MiB
On-chip memory used for caching model parameters: 5.52MiB
On-chip memory remaining for caching model parameters: 2.50KiB
Off-chip memory used for streaming uncached model parameters: 49.61MiB
Number of Edge TPU subgraphs: 1
Total number of operations: 395
Operation log: inception_resnet_v2_edgetpu.log
See the operation log file for individual operation details.
Compilation child process completed within timeout period.
Compilation succeeded! 

Edge TPU Compiler version 16.0.384591198
Input: inception_resnet_v2.tflite
Output:

In [21]:
%time testEdgeTPUCompilation(InceptionV3, None)

96116736/96112376 [==============================] - 2s 0us/step
Model name: inception_v3
Input shape
INFO:tensorflow:Froze 378 variables.
INFO:tensorflow:Converted 378 variables to const ops.
Edge TPU Compiler version 16.0.384591198
Started a compilation timeout timer of 180 seconds.

Model compiled successfully in 5223 ms.

Input model: inception_v3.tflite
Input size: 23.21MiB
Output model: inception_v3_edgetpu.tflite
Output size: 23.99MiB
On-chip memory used for caching model parameters: 5.53MiB
On-chip memory remaining for caching model parameters: 256.00B
Off-chip memory used for streaming uncached model parameters: 17.97MiB
Number of Edge TPU subgraphs: 1
Total number of operations: 162
Operation log: inception_v3_edgetpu.log
See the operation log file for individual operation details.
Compilation child process completed within timeout period.
Compilation succeeded! 

Edge TPU Compiler version 16.0.384591198
Input: inception_v3.tflite
Output: inception_v3_edgetpu.tflite

Operator

In [22]:
%time testEdgeTPUCompilation(NASNetLarge, None)

359751680/359748576 [==============================] - 15s 0us/step
Model name: NASNet
Input shape
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Froze 1546 variables.
INFO:tensorflow:Converted 1546 variables to const ops.
Edge TPU Compiler version 16.0.384591198
Started a compilation timeout timer of 180 seconds.

Model compiled successfully in 558 ms.

Input model: NASNet.tflite
Input size: 88.37MiB
Output model: NASNet_edgetpu.tflite
Output size: 88.43MiB
On-chip memory used for caching model parameters: 6.00KiB
On-chip memory remaining for caching model parameters: 7.67MiB
Off-chip memory used for streaming uncached model parameters: 0.00B
Number of Edge TPU subgraphs: 1
Total number of operations: 1333
Operation log: NASNet_edgetpu.log

Model successfully compiled but not all operations are supported by the Edge TPU. A percentage of the model will instead run on the CPU, which is slower. If poss

In [23]:
%time testEdgeTPUCompilation(NASNetMobile, None)

24231936/24227760 [==============================] - 0s 0us/step
Model name: NASNet
Input shape
INFO:tensorflow:Froze 1126 variables.
INFO:tensorflow:Converted 1126 variables to const ops.
Edge TPU Compiler version 16.0.384591198
Started a compilation timeout timer of 180 seconds.

Model compiled successfully in 98 ms.

Input model: NASNet.tflite
Input size: 5.94MiB
Output model: NASNet_edgetpu.tflite
Output size: 5.96MiB
On-chip memory used for caching model parameters: 3.00KiB
On-chip memory remaining for caching model parameters: 7.90MiB
Off-chip memory used for streaming uncached model parameters: 0.00B
Number of Edge TPU subgraphs: 1
Total number of operations: 991
Operation log: NASNet_edgetpu.log

Model successfully compiled but not all operations are supported by the Edge TPU. A percentage of the model will instead run on the CPU, which is slower. If possible, consider updating your model to use only operations supported by the Edge TPU. For details, visit g.co/coral/model-reqs

In [24]:
%time testEdgeTPUCompilation(ResNet101, None)

179650560/179648224 [==============================] - 3s 0us/step
Model name: resnet101
Input shape
INFO:tensorflow:Froze 626 variables.
INFO:tensorflow:Converted 626 variables to const ops.
Edge TPU Compiler version 16.0.384591198
Started a compilation timeout timer of 180 seconds.

Model compiled successfully in 9061 ms.

Input model: resnet101.tflite
Input size: 43.85MiB
Output model: resnet101_edgetpu.tflite
Output size: 43.43MiB
On-chip memory used for caching model parameters: 7.14MiB
On-chip memory remaining for caching model parameters: 512.00B
Off-chip memory used for streaming uncached model parameters: 35.90MiB
Number of Edge TPU subgraphs: 1
Total number of operations: 145
Operation log: resnet101_edgetpu.log
See the operation log file for individual operation details.
Compilation child process completed within timeout period.
Compilation succeeded! 

Edge TPU Compiler version 16.0.384591198
Input: resnet101.tflite
Output: resnet101_edgetpu.tflite

Operator                

In [25]:
%time testEdgeTPUCompilation(ResNet101V2, None)

179519488/179518384 [==============================] - 2s 0us/step
Model name: resnet101v2
Input shape
INFO:tensorflow:Froze 544 variables.
INFO:tensorflow:Converted 544 variables to const ops.
Edge TPU Compiler version 16.0.384591198
Started a compilation timeout timer of 180 seconds.

Model compiled successfully in 9854 ms.

Input model: resnet101v2.tflite
Input size: 43.94MiB
Output model: resnet101v2_edgetpu.tflite
Output size: 44.43MiB
On-chip memory used for caching model parameters: 6.60MiB
On-chip memory remaining for caching model parameters: 1.00KiB
Off-chip memory used for streaming uncached model parameters: 37.36MiB
Number of Edge TPU subgraphs: 1
Total number of operations: 249
Operation log: resnet101v2_edgetpu.log
See the operation log file for individual operation details.
Compilation child process completed within timeout period.
Compilation succeeded! 

Edge TPU Compiler version 16.0.384591198
Input: resnet101v2.tflite
Output: resnet101v2_edgetpu.tflite

Operator    

In [26]:
%time testEdgeTPUCompilation(ResNet152, None)

242900992/242900224 [==============================] - 3s 0us/step
Model name: resnet152
Input shape
INFO:tensorflow:Froze 932 variables.
INFO:tensorflow:Converted 932 variables to const ops.
Edge TPU Compiler version 16.0.384591198
Started a compilation timeout timer of 180 seconds.

Model compiled successfully in 12408 ms.

Input model: resnet152.tflite
Input size: 59.36MiB
Output model: resnet152_edgetpu.tflite
Output size: 58.71MiB
On-chip memory used for caching model parameters: 7.14MiB
On-chip memory remaining for caching model parameters: 512.00B
Off-chip memory used for streaming uncached model parameters: 51.04MiB
Number of Edge TPU subgraphs: 1
Total number of operations: 213
Operation log: resnet152_edgetpu.log
See the operation log file for individual operation details.
Compilation child process completed within timeout period.
Compilation succeeded! 

Edge TPU Compiler version 16.0.384591198
Input: resnet152.tflite
Output: resnet152_edgetpu.tflite

Operator               

In [27]:
%time testEdgeTPUCompilation(ResNet152V2, None)

242753536/242745792 [==============================] - 4s 0us/step
Model name: resnet152v2
Input shape
INFO:tensorflow:Froze 816 variables.
INFO:tensorflow:Converted 816 variables to const ops.
Edge TPU Compiler version 16.0.384591198
Started a compilation timeout timer of 180 seconds.

Model compiled successfully in 13632 ms.

Input model: resnet152v2.tflite
Input size: 59.50MiB
Output model: resnet152v2_edgetpu.tflite
Output size: 60.20MiB
On-chip memory used for caching model parameters: 6.60MiB
On-chip memory remaining for caching model parameters: 1.00KiB
Off-chip memory used for streaming uncached model parameters: 52.94MiB
Number of Edge TPU subgraphs: 1
Total number of operations: 368
Operation log: resnet152v2_edgetpu.log
See the operation log file for individual operation details.
Compilation child process completed within timeout period.
Compilation succeeded! 

Edge TPU Compiler version 16.0.384591198
Input: resnet152v2.tflite
Output: resnet152v2_edgetpu.tflite

Operator   

In [28]:
%time testEdgeTPUCompilation(ResNet50, None)

102973440/102967424 [==============================] - 1s 0us/step
Model name: resnet50
Input shape
INFO:tensorflow:Froze 320 variables.
INFO:tensorflow:Converted 320 variables to const ops.
Edge TPU Compiler version 16.0.384591198
Started a compilation timeout timer of 180 seconds.

Model compiled successfully in 5162 ms.

Input model: resnet50.tflite
Input size: 25.06MiB
Output model: resnet50_edgetpu.tflite
Output size: 24.91MiB
On-chip memory used for caching model parameters: 7.14MiB
On-chip memory remaining for caching model parameters: 512.00B
Off-chip memory used for streaming uncached model parameters: 17.54MiB
Number of Edge TPU subgraphs: 1
Total number of operations: 77
Operation log: resnet50_edgetpu.log
See the operation log file for individual operation details.
Compilation child process completed within timeout period.
Compilation succeeded! 

Edge TPU Compiler version 16.0.384591198
Input: resnet50.tflite
Output: resnet50_edgetpu.tflite

Operator                       

In [29]:
%time testEdgeTPUCompilation(ResNet50V2, None)

102875136/102869336 [==============================] - 1s 0us/step
Model name: resnet50v2
Input shape
INFO:tensorflow:Froze 272 variables.
INFO:tensorflow:Converted 272 variables to const ops.
Edge TPU Compiler version 16.0.384591198
Started a compilation timeout timer of 180 seconds.

Model compiled successfully in 5537 ms.

Input model: resnet50v2.tflite
Input size: 25.11MiB
Output model: resnet50v2_edgetpu.tflite
Output size: 25.37MiB
On-chip memory used for caching model parameters: 6.60MiB
On-chip memory remaining for caching model parameters: 1.00KiB
Off-chip memory used for streaming uncached model parameters: 18.49MiB
Number of Edge TPU subgraphs: 1
Total number of operations: 130
Operation log: resnet50v2_edgetpu.log
See the operation log file for individual operation details.
Compilation child process completed within timeout period.
Compilation succeeded! 

Edge TPU Compiler version 16.0.384591198
Input: resnet50v2.tflite
Output: resnet50v2_edgetpu.tflite

Operator          

In [30]:
%time testEdgeTPUCompilation(VGG16, None)

553467904/553467096 [==============================] - 9s 0us/step
Model name: vgg16
Input shape
INFO:tensorflow:Froze 32 variables.
INFO:tensorflow:Converted 32 variables to const ops.
Edge TPU Compiler version 16.0.384591198
Started a compilation timeout timer of 180 seconds.

Model compiled successfully in 23654 ms.

Input model: vgg16.tflite
Input size: 132.09MiB
Output model: vgg16_edgetpu.tflite
Output size: 132.32MiB
On-chip memory used for caching model parameters: 5.33MiB
On-chip memory remaining for caching model parameters: 170.25KiB
Off-chip memory used for streaming uncached model parameters: 126.78MiB
Number of Edge TPU subgraphs: 1
Total number of operations: 24
Operation log: vgg16_edgetpu.log
See the operation log file for individual operation details.
Compilation child process completed within timeout period.
Compilation succeeded! 

Edge TPU Compiler version 16.0.384591198
Input: vgg16.tflite
Output: vgg16_edgetpu.tflite

Operator                       Count      Sta

In [31]:
%time testEdgeTPUCompilation(VGG19, None)

574717952/574710816 [==============================] - 12s 0us/step
Model name: vgg19
Input shape
INFO:tensorflow:Froze 38 variables.
INFO:tensorflow:Converted 38 variables to const ops.
Edge TPU Compiler version 16.0.384591198
Started a compilation timeout timer of 180 seconds.

Model compiled successfully in 24589 ms.

Input model: vgg19.tflite
Input size: 137.19MiB
Output model: vgg19_edgetpu.tflite
Output size: 137.41MiB
On-chip memory used for caching model parameters: 5.33MiB
On-chip memory remaining for caching model parameters: 170.25KiB
Off-chip memory used for streaming uncached model parameters: 131.86MiB
Number of Edge TPU subgraphs: 1
Total number of operations: 27
Operation log: vgg19_edgetpu.log
See the operation log file for individual operation details.
Compilation child process completed within timeout period.
Compilation succeeded! 

Edge TPU Compiler version 16.0.384591198
Input: vgg19.tflite
Output: vgg19_edgetpu.tflite

Operator                       Count      St

In [32]:
%time testEdgeTPUCompilation(Xception, None)

91889664/91884032 [==============================] - 2s 0us/step
Model name: xception
Input shape
INFO:tensorflow:Froze 236 variables.
INFO:tensorflow:Converted 236 variables to const ops.
Edge TPU Compiler version 16.0.384591198
Started a compilation timeout timer of 180 seconds.

Model compiled successfully in 534 ms.

Input model: xception.tflite
Input size: 22.84MiB
Output model: xception_edgetpu.tflite
Output size: 23.10MiB
On-chip memory used for caching model parameters: 101.50KiB
On-chip memory remaining for caching model parameters: 6.45MiB
Off-chip memory used for streaming uncached model parameters: 0.00B
Number of Edge TPU subgraphs: 1
Total number of operations: 128
Operation log: xception_edgetpu.log

Model successfully compiled but not all operations are supported by the Edge TPU. A percentage of the model will instead run on the CPU, which is slower. If possible, consider updating your model to use only operations supported by the Edge TPU. For details, visit g.co/coral